# This notebook expressed the training progress for CNN-LSTM version 2 model

## Data Import

In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np

In [2]:
# Suppress future warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
# Add project root to Python path for imports
import os
project_root = os.path.abspath('..')  # Go up one level from notebooks to project root
if project_root not in sys.path:
    sys.path.append(project_root)

# Load processed data
data_path = os.path.join(project_root, 'data', 'processed', 'emotion.csv')
data = pd.read_csv(data_path)
data

,0,1,2,3,4,5,6,7,8,9,...,2367,2368,2369,2370,2371,2372,2373,2374,2375,Emotions
0,0.173828,0.264648,0.372559,0.486328,0.625488,0.681152,0.720703,0.651855,0.562988,0.559082,...,0.766315,0.755441,0.743671,0.731043,0.717561,0.703260,0.688124,0.672208,0.655518,neutral
1,0.231934,0.357422,0.482910,0.472656,0.480469,0.484375,0.491699,0.490723,0.500977,0.503418,...,1.594981,-1.356783,3.955091,1.570934,1.334768,0.901523,-0.066584,-6.374361,-7.761413,neutral
2,0.256348,0.354492,0.471191,0.414062,0.378418,0.373535,0.354492,0.408691,0.466797,0.522461,...,0.006713,0.006601,0.006483,0.006356,0.006218,0.006066,0.005919,0.005759,0.005588,neutral
3,0.384766,0.576172,0.764160,0.778809,0.742188,0.752930,0.787109,0.799316,0.700195,0.590332,...,6.918442,6.534257,6.109815,5.613272,5.033775,4.383093,3.692577,3.005888,2.370324,neutral
4,0.247070,0.389648,0.564941,0.642578,0.665527,0.684570,0.676270,0.666992,0.648926,0.595703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48643,0.025391,0.035645,0.043457,0.076660,0.116211,0.119141,0.121094,0.081543,0.038574,0.032715,...,2.149720,-1.269249,8.457794,-0.209345,3.439721,10.229767,14.676046,4.467748,4.202962,surprise
48644,0.000000,0.000000,0.006348,0.016113,0.025879,0.034180,0.036621,0.033691,0.032715,0.032227,...,-4.478354,-5.067346,-4.775791,0.778204,5.260514,-0.345362,-1.215722,8.969276,7.940011,surprise
48645,0.078125,0.125977,0.163574,0.132324,0.104980,0.066406,0.065918,0.074707,0.076660,0.081055,...,-5.975361,-4.235834,-2.939499,-1.558753,0.182943,2.841793,4.177210,4.016248,0.110962,surprise
48646,0.004883,0.015625,0.026367,0.036621,0.042969,0.041504,0.043457,0.043457,0.041504,0.042480,...,-17.944071,-13.947256,-3.418745,6.305834,9.191437,-7.628603,1.824499,13.876075,-1.117508,surprise


In [4]:
# Check if data has any NaN values
print(data.isna().any())

0           False
1           False
2           False
3           False
4           False
            ...  
2372         True
2373         True
2374         True
2375         True
Emotions    False
Length: 2377, dtype: bool


In [5]:
# In preprocessing data, there are so many problems which can create NaN values, so in this situation, we just replace them with 0 to make it simple.
Emotions= data.fillna(0)
print(Emotions.isna().any())
Emotions.shape

0           False
1           False
2           False
3           False
4           False
            ...  
2372        False
2373        False
2374        False
2375        False
Emotions    False
Length: 2377, dtype: bool


(48648, 2377)

In [6]:
np.sum(Emotions.isna())

0           0
1           0
2           0
3           0
4           0
           ..
2372        0
2373        0
2374        0
2375        0
Emotions    0
Length: 2377, dtype: int64

## Data Preprocess

In [7]:
# Taking all rows and all cols without last col for X which include features
# Taking last col for Y, which include the emotions

X,Y = Emotions.iloc[:,:-1].values, Emotions['Emotions'].values

In [8]:
# As this is a multiclass classification problem onehotencoding our Y
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [9]:
print('X shape: ',X.shape)
print('Y shape: ',Y.shape)

X shape:  (48648, 2376)
Y shape:  (48648, 7)


## Train mode

In [10]:
# Define const values
LEARNING_RATE = 0.01
MOMENTUM_RATE = 0.9
EPOCHS = 50
PATIENCE = 5

In [11]:
import torch
from torch.utils.data import TensorDataset, random_split

# Convert to PyTorch tensors
full_dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32))

In [12]:
# Split the dataset
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

sizes = [train_size, val_size, test_size]

train_dataset, val_dataset, test_dataset = random_split(dataset=full_dataset, lengths=sizes, generator=torch.Generator().manual_seed(42))

print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

Train dataset size: 38918
Validation dataset size: 4864
Test dataset size: 4866


In [13]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

# DataLoader for training dataset (Training): BẬT shuffle
train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

# DataLoader for validation dataset (Validation): TẮT shuffle
val_loader = DataLoader(
    dataset=val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

# DataLoader for test dataset (Test): TẮT shuffle
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

In [14]:
# Define model
from src.model.CNN_LSTM_model import create_model

model = create_model(input_size=X.shape[-1], n_emotions=Y.shape[-1])

In [15]:
# Define optimizer
import torch.optim as optim

optimizer = optim.SGD(
    params=model.parameters(), 
    lr=LEARNING_RATE, 
    momentum=MOMENTUM_RATE,
    nesterov=True # Enable Nesterov momentum
)

# Define loss function
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [16]:
# Define reduce learning rate on plateau scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(
    optimizer,
    mode='max',        
    factor=0.5,        
    patience=3,
    min_lr=1e-5,
)

In [18]:
# Start training model
best_val_acc = 0
counter = 0
train_acc_list = []
val_acc_list = []

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        output = model(x)
        
        # Convert one-hot y to class indices for CrossEntropyLoss
        y_indices = torch.argmax(y, dim=1) # [0,1,0,0] -> 1
        
        loss = criterion(output, y_indices)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        # Calculate training accuracy
        pred = torch.argmax(output, dim=1)
        train_correct += (pred == y_indices).sum().item()
        train_total += y.size(0)
    
    # Calculate average training loss and accuracy
    avg_train_loss = train_loss / len(train_loader)
    train_acc = train_correct / train_total
    train_acc_list.append(train_acc)
    # Validation phase
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for x, y in val_loader:
            output = model(x)
            
            # Convert one-hot y to class indices for loss calculation
            y_indices = torch.argmax(y, dim=1)
            
            loss = criterion(output, y_indices)
            val_loss += loss.item()
            
            # Calculate validation accuracy
            pred = torch.argmax(output, dim=1)
            val_correct += (pred == y_indices).sum().item()
            val_total += y.size(0)
    
    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_acc = val_correct / val_total
    val_acc_list.append(val_acc)
    # Print epoch results
    print(f'Epoch [{epoch+1}/{EPOCHS}]:')
    print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}')
    
    # Learning rate scheduling
    scheduler.step(val_acc)
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
        # Save best model
        torch.save(model.state_dict(), '../model/best_model.pth')
        print(f'  New best validation accuracy: {best_val_acc:.4f}')
    else:
        counter += 1
        
    if counter >= PATIENCE:
        print(f'Early stopping at epoch {epoch+1}')
        break
        
print(f'Training completed! Best validation accuracy: {best_val_acc:.4f}')

Epoch [1/50]:
  Train Loss: 1.2754, Train Acc: 0.5017
  Val Loss: 1.4430, Val Acc: 0.4305
  New best validation accuracy: 0.4305
Epoch [2/50]:
  Train Loss: 1.1946, Train Acc: 0.5325
  Val Loss: 1.3103, Val Acc: 0.4928
  New best validation accuracy: 0.4928
Epoch [2/50]:
  Train Loss: 1.1946, Train Acc: 0.5325
  Val Loss: 1.3103, Val Acc: 0.4928
  New best validation accuracy: 0.4928
Epoch [3/50]:
  Train Loss: 1.1337, Train Acc: 0.5534
  Val Loss: 1.2034, Val Acc: 0.5403
  New best validation accuracy: 0.5403
Epoch [3/50]:
  Train Loss: 1.1337, Train Acc: 0.5534
  Val Loss: 1.2034, Val Acc: 0.5403
  New best validation accuracy: 0.5403
Epoch [4/50]:
  Train Loss: 1.0823, Train Acc: 0.5793
  Val Loss: 1.2553, Val Acc: 0.5148
Epoch [4/50]:
  Train Loss: 1.0823, Train Acc: 0.5793
  Val Loss: 1.2553, Val Acc: 0.5148
Epoch [5/50]:
  Train Loss: 1.0356, Train Acc: 0.5978
  Val Loss: 1.1518, Val Acc: 0.5432
  New best validation accuracy: 0.5432
Epoch [5/50]:
  Train Loss: 1.0356, Train Acc:

In [19]:
model.eval()
test_loss = 0
test_correct = 0
test_total = 0

with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        
        # Convert one-hot y to class indices for loss calculation
        y_indices = torch.argmax(y, dim=1)
        
        loss = criterion(output, y_indices)
        test_loss += loss.item()
        
        # Calculate validation accuracy
        pred = torch.argmax(output, dim=1)
        test_correct += (pred == y_indices).sum().item()
        test_total += y.size(0)

# Calculate average validation loss and accuracy
avg_test_loss = test_loss / len(val_loader)
test_acc = test_correct / test_total
print(f'Average test loss: {avg_test_loss}')
print(f'Test accuracy: {test_acc}')

Average test loss: 1.434282020213064
Test accuracy: 0.6510480887792849
